In [1]:
import pandas as pd
# import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns
import sys
import os
import numpy as np
from pandas.api.types import is_numeric_dtype




In [2]:
df = pd.read_csv(os.path.abspath(os.path.join('../'))+"/data/cleaned_data.csv")

In [5]:
df.columns

Index(['Bearer Id', 'Start', 'Start ms', 'End', 'End ms', 'Dur. (ms)', 'IMSI',
       'MSISDN/Number', 'IMEI', 'Last Location Name', 'Avg RTT DL (ms)',
       'Avg RTT UL (ms)', 'Avg Bearer TP DL (kbps)', 'Avg Bearer TP UL (kbps)',
       'TCP DL Retrans. Vol (Bytes)', 'TCP UL Retrans. Vol (Bytes)',
       'DL TP < 50 Kbps (%)', '50 Kbps < DL TP < 250 Kbps (%)',
       '250 Kbps < DL TP < 1 Mbps (%)', 'DL TP > 1 Mbps (%)',
       'UL TP < 10 Kbps (%)', '10 Kbps < UL TP < 50 Kbps (%)',
       '50 Kbps < UL TP < 300 Kbps (%)', 'UL TP > 300 Kbps (%)',
       'HTTP DL (Bytes)', 'HTTP UL (Bytes)', 'Activity Duration DL (ms)',
       'Activity Duration UL (ms)', 'Dur. (ms).1', 'Handset Manufacturer',
       'Handset Type', 'Nb of sec with 125000B < Vol DL',
       'Nb of sec with 1250B < Vol UL < 6250B',
       'Nb of sec with 31250B < Vol DL < 125000B',
       'Nb of sec with 37500B < Vol UL',
       'Nb of sec with 6250B < Vol DL < 31250B',
       'Nb of sec with 6250B < Vol UL < 37500B',


Tracking user engagement based on the following metrics
- sessions frequency 
- the duration of the session 
- the sessions total traffic (download and upload (bytes))

Task 2.1 - Based on the above submit python script and slide :
Aggregate the above metrics per customer id (MSISDN) and report the top 10 customers per engagement metric 



In [27]:
# session frequency per user
print("Top 10 users ranked based on Session frequency")
session_freq = df.groupby('MSISDN/Number')["Bearer Id"].count().reset_index()
session_freq.rename(columns = {'Bearer Id':'session_freq'},inplace=True)
session_freq.sort_values(by = "session_freq",ascending=False).head(10)

Top 10 users ranked based on Session frequency


,MSISDN/Number,session_freq
13526,3.362632e+10,18
13180,3.362578e+10,17
6437,3.361489e+10,17
37052,3.365973e+10,16
76363,3.367588e+10,15
92923,3.376054e+10,15
65118,3.366716e+10,13
92577,3.376041e+10,12
1279,3.360452e+10,12
666,3.360313e+10,12


In [29]:
# Session Duration
print("Top 10 users ranked based on Session duration")
session_dur = df.groupby('MSISDN/Number')["Dur. (ms)"].sum().reset_index()
session_dur.rename(columns = {'Dur. (ms)':'session_dur'},inplace=True)
session_dur.sort_values(by = "session_dur",ascending=False).head(10)



Top 10 users ranked based on Session duration


,MSISDN/Number,session_dur
13526,3.362632e+10,2339451.00
13180,3.362578e+10,2251314.25
6437,3.361489e+10,2223675.00
37052,3.365973e+10,2080839.75
92923,3.376054e+10,1978671.50
76363,3.367588e+10,1974039.50
65118,3.366716e+10,1721593.25
666,3.360313e+10,1589163.00
13994,3.362708e+10,1589163.00
1279,3.360452e+10,1589163.00


In [30]:
# creating total volume from Total UL (Bytes) and Total DL (Bytes)
df["Total volume(Bytes)"] = df['Total UL (Bytes)']+df['Total DL (Bytes)']

In [31]:
# Session upload traffic 
print("Top 10 users ranked based on Session traffic")

session_traffic = df.groupby('MSISDN/Number')["Total volume(Bytes)"].sum().reset_index()
session_traffic.rename(columns = {'Total volume(Bytes)':'session_traffic'},inplace=True)
session_traffic.sort_values(by = "session_traffic",ascending=False).head(10)


Top 10 users ranked based on Session traffic


,MSISDN/Number,session_traffic
6437,3.361489e+10,8.501063e+09
92923,3.376054e+10,8.449641e+09
13180,3.362578e+10,8.170284e+09
13526,3.362632e+10,8.005959e+09
76363,3.367588e+10,7.782932e+09
37052,3.365973e+10,7.567114e+09
63028,3.366646e+10,6.712227e+09
57241,3.366471e+10,6.679455e+09
92577,3.376041e+10,6.469516e+09
86455,3.369879e+10,6.204933e+09


In [34]:
# join all the columns
user_engagement = pd.merge(session_dur, session_freq, on='MSISDN/Number')
user_engagement = pd.merge(user_engagement, session_traffic, on='MSISDN/Number')
user_engagement

,MSISDN/Number,session_dur,session_freq,session_traffic
0,3.360100e+10,116720.00,1,7.017587e+08
1,3.360100e+10,132430.25,1,2.792113e+08
2,3.360100e+10,132430.25,1,5.959665e+08
3,3.360101e+10,57440.50,1,4.352152e+08
4,3.360101e+10,114881.00,2,1.300318e+09
...,...,...,...,...
106851,3.379000e+10,57440.50,1,6.989276e+08
106852,3.379000e+10,132430.25,1,4.803073e+08
106853,3.197021e+12,132430.25,1,2.804027e+08
106854,3.370000e+14,132430.25,1,5.886692e+08


Normalize each engagement metric and run a k-means (k=3) to classify customers in three groups of engagement. 


Compute the minimum, maximum, average & total non- normalized metrics for each cluster. Interpret your results visually with accompanying text explaining your findings.
Aggregate user total traffic per application and derive the top 10 most engaged users per application
Plot the top 3 most used applications using appropriate charts.  
Using k-means clustering algorithm, group users in k engagement clusters based on the engagement metrics: 
What is the optimized value of k (use elbow method for this)?  
Interpret your findings. 